<a href="https://colab.research.google.com/github/nptan2005/spark401_colab/blob/main/notebooks/spark_level_3_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install java

In [ ]:
!apt-get install -y openjdk-17-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jre session-migration x11-utils
Suggested packages:
  gvfs libxt-doc openjdk-17-demo openjdk-17-source visualvm mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jdk openjdk-17-jr

# Install Spark

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz

--2025-12-30 07:54:41--  https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548955321 (524M) [application/x-gzip]
Saving to: ‘spark-4.0.1-bin-hadoop3.tgz’

spark-4.0.1-bin-had 100%[===================>] 523.52M  15.7MB/s    in 36s     

2025-12-30 07:55:18 (14.4 MB/s) - ‘spark-4.0.1-bin-hadoop3.tgz’ saved [548955321/548955321]



# Setup environment

In [4]:
# ===============================
# Spark 4.0.1 Setup (REQUIRED)
# ===============================
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark401-Training") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("Spark version:", spark.version)

Spark version: 4.0.1


# 🚀 Spark Level 3 – Internals

## Bài 1: DAG – Stage – Task (CỐT LÕI CỦA SPARK)

❗ Nếu hiểu phần này → bạn vượt 90% người dùng Spark

---

### 1️⃣ Tổng quan kiến trúc thực thi Spark

**Spark Execution Flow:**

```code
Code
 ↓
Logical Plan
 ↓
Physical Plan
 ↓
DAG
 ↓
Stages
 ↓
Tasks
```

---

### 2️⃣ DAG là gì?

**DAG (Directed Acyclic Graph) là:**
- Đồ thị biểu diễn toàn bộ pipeline tính toán
- Các node = transformation
- Các cạnh = dependency
- Không có vòng lặp

> 📌 DAG được tạo KHI CÓ ACTION

---

### 3️⃣ Stage là gì?

**Stage là:**
- Một tập các task có thể chạy song song
- Bị chia cắt bởi SHUFFLE

**Quy tắc VÀNG:**

> Mỗi lần shuffle → tạo stage mới

** Ví dụ:**

```python
df.filter(...).select(...)   # Stage 1
df.groupBy(...).count()      # Stage 2 (shuffle)
```

---

### 4️⃣ Task là gì?

**Task là:**
- Đơn vị nhỏ nhất Spark có thể thực thi
- Một task xử lý một partition
- Chạy trên một executor

> 📌 Số task = số partition trong stage

---

### 5️⃣ Ví dụ THỰC TẾ

```python
df = spark.read.parquet("data")

df2 = df.filter("amount > 1000")      # Transformation
df3 = df2.groupBy("customer_id").sum("amount")  # Shuffle

df3.show()   # Action
```

#### DAG sẽ là:

**Stage 0:**
- Read
- Filter

**Stage 1:**
- Exchange (shuffle)
- HashAggregate

---

### 6️⃣ Vì sao hiểu Stage quan trọng?

|**Vấn đề**|**Liên quan**|
|----------|-------------|
|Job chạy chậm|Stage skew|
|Executor OOM|Task quá lớn|
|Shuffle nhiều|Stage split|
|Tuning Spark|Stage-based|

---

## 🧠 CÂU HỎI BẮT BUỘC – LEVEL 3 / BÀI 1

Hãy trả lời:
1.	Khi nào DAG được tạo?
> DAG được tạo khi có **ACTION**
2.	Điều gì quyết định số stage?
> shuffle
3.	Số task phụ thuộc vào yếu tố nào?
> partition trong stage
4.	Vì sao Spark không thể chia nhỏ một task đang chạy?
> Vì task là 1 đơn vị nhỏ nhất
5.	Vì sao giảm shuffle giúp Spark nhanh hơn?
> Tạo stage mới

